In [ ]:
import numpy
import pandas as pd

from geopy.geocoders import Nominatim


In [ ]:
# Import/ install all packages
import numpy
import pandas as pd

from geopy.geocoders import Nominatim

!pip install arcgis
from arcgis.geocoding import geocode
from arcgis.gis import GIS

!pip install haversine
import haversine as hs
from haversine import Unit

!pip install ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

     |████████████████████████████████| 3.7 MB 5.1 MB/s 
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached cryptography-36.0.2-cp36-abi3-manylinux_2_24_x86_64.whl (3.6 MB)
  Using cached keyring-21.7.0-py3-none-any.whl (32 kB)
  Using cached lerc-0.1.0-py3-none-any.whl
  Using cached ujson-5.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43 kB)
  Using cached jupyterlab-3.3.2-py3-none-any.whl (8.7 MB)
  Using cached python_certifi_win32-1.6-py2.py3-none-any.whl (7.2 kB)
  Using cached pyshp-2.2.0-py3-none-any.whl (44 kB)
  Using cached geomet-0.3.0-py3-none-any.whl (28 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached requests_ntlm-1.1.0-py2.py3-none-any.whl (5.7 kB)
  Using cached requests_gssapi-1.2.3-py2.py3-none-any.whl
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 428 kB 48.9 MB/s 
     |███████████

In [ ]:
# Function to append coordinates

geolocator = Nominatim(user_agent="my-application")

longitude_arr = []
latitude_arr = []
new_houses_arr = []
coord_arr = []

def append_coordinates(df):
  display(df)

  for i in df['Houses']:
    try:
      location = geolocator.geocode(i)
      new_houses_arr.append(location.address)
      longitude_arr.append(location.longitude)
      latitude_arr.append(location.latitude)
      ### coords are in opposite order below
      coord_arr.append((location.longitude,location.latitude))
    except:
      pass

  d = {'new_houses':new_houses_arr,'longitude':longitude_arr,'latitude':latitude_arr, 'coord':coord_arr}
  df_with_coordinates = pd.DataFrame(d)
  display(df_with_coordinates)
  return df_with_coordinates


In [ ]:
# Function which:
# Finds public places where volunteers can congregate
# The display shows parks in given city

def find_public_locations(city,type_of_place):
  gis = GIS()
  city_off = geocode(city)[0]
  places = geocode(None, city_off['extent'], category=type_of_place, max_locations=100)
  for place in places:
      print(place['attributes']['PlaceName'])
      print(place['location'])
  return places

In [ ]:
# Function which finds the top n closest houses to the public place
import numpy as np
from datascience import *

def find_n_closest(df_with_coordinates,chosen_park,n):
  df_trial = df_with_coordinates
  df_tri = Table().with_columns('new houses',new_houses_arr,'longitude',longitude_arr,'latitude',latitude_arr, 'coord',coord_arr)

  distances = make_array()
  for row in df_tri.rows:
    loc1 = (row.item('latitude'),row.item('longitude'))
    loc2 = (chosen_park['location']['y'],chosen_park['location']['x'])
    distances = np.append(distances, hs.haversine(loc1,loc2,unit=Unit.MILES))

  df_tri_with_distances = df_tri.with_column('distance from park',distances)
  df_tri_sorted = df_tri_with_distances.sort('distance from park')
  df_n_stops = df_tri_sorted.take(np.arange(0,n))
  df_n_stops.show()
  return df_n_stops


/usr/local/lib/python3.7/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [ ]:
# Function that makes the public place a stop

def append_place(df_10_stops):
  houses_with_park = np.append(df_10_stops.column("new houses"), chosen_park['address'])
  longitude_with_park = np.append(df_10_stops.column("longitude"),chosen_park['location']['x'])
  latitude_with_park = np.append(df_10_stops.column("latitude"),chosen_park['location']['y'])
  dist_from_park_with_park = np.append(df_10_stops.column("distance from park"),0)

  df_with_park = Table().with_columns("locations",houses_with_park,
                                      "longitude", longitude_with_park,
                                      "latitude", latitude_with_park)
  df_with_park_sorted = df_with_park.sort('locations', descending= True)
  df_with_park_sorted.show()
  return df_with_park_sorted

In [ ]:
# Function to take dataset and create 2-D array with all pairs' distances

def find_distances(my_row):
  dist = []
  loc1 = (my_row.item('latitude'),my_row.item('longitude'))
  for other_row in df_with_park_sorted.rows:
    loc2 = (other_row.item('latitude'),other_row.item('longitude'))
    dist.append(hs.haversine(loc1,loc2,unit=Unit.MILES))
  return dist

def make_2d_array(df_with_park_sorted):
  dist_2d_arr = []

  for my_row in df_with_park_sorted.rows:
    dist_2d_arr.append(find_distances(my_row))

  return dist_2d_arr

In [ ]:
# Import cleaned dataset of Houses with solar panels in Tooele, UT
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Houses with solar panels- shreya - Tooele Summary.csv']))

In [ ]:
df_with_coordinates = append_coordinates(df)

In [ ]:
parks = find_public_locations("Tooele,UT","Park")

In [ ]:
chosen_park = parks[1]
print(chosen_park)

In [ ]:
df_n_stops = find_n_closest(df_with_coordinates,chosen_park,10)

In [ ]:
df_with_park_sorted = append_place(df_n_stops)

In [ ]:
dist_2d_arr = make_2d_array(df_with_park_sorted)
dist_2d_arr

In [ ]:
# Using Google OR Tools, utilize travelling salesman problem to find the route volunteers should take for visiting houses, starting and ending with the public place.

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist_2d_arr
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    stops_in_order = ""
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        stops_in_order = stops_in_order + df_with_park_sorted.take(index).column('locations').item(0) + " -> \n"
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    stops_in_order += df_with_park_sorted.take(0).column('locations').item(0)
    print("This is the optimal route:")
    print(stops_in_order)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


if __name__ == '__main__':
    main()

In [ ]:
find_public_locations("Tooele,UT","Park")

NameError: ignored

**Mapping**

In [ ]:
#install folium packages
!pip install folium
import folium
import folium.plugins

In [ ]:
#create map of Toole, Utah
map = folium.Map(location=[40.5308, -112.2983], min_zoom =14, max_zoom = 18)
map

**NOTE** :
this might fail if the coordinates are not in the correct order (lat,lon)

In [ ]:
#create marker for each place & mark a path
def markers_and_path(series_of_coords, m):
  for each in series_of_coords:
    folium.Marker(each).add_to(m)
  #add lines
  folium.PolyLine(series_of_coords, color="red", weight=2.5, opacity=1).add_to(m)
  display(m)
coordinates = ... #series of coorinates sorted by order of shortest path
markers_and_path(coordinates, map)